
# Formulário de Avaliação Inicial do Corredor

Este notebook funciona como um **formulário interativo** para coleta de dados de um corredor.  
Ao preencher os campos e clicar em **Salvar dados e classificar**, o notebook irá:

1. Construir um objeto Python seguindo a **estrutura do JSON Schema** definido previamente.
2. Salvar os dados em um arquivo `corredor_dados.json` no diretório atual.
3. Gerar um **rótulo automático** de classificação do corredor:  
   - `iniciante`  
   - `intermediario`  
   - `avancado`  

A classificação é baseada principalmente em:

- `volume_semanal_km`
- `melhor_5k_segundos`

> **Obs.:** Campos não preenchidos em tempos de prova serão salvos como `null` no JSON.


In [ ]:

import json
from ipywidgets import (
    VBox, HBox, Label, IntText, BoundedIntText, FloatText, BoundedFloatText,
    Dropdown, Checkbox, Button, Output
)
from IPython.display import display, clear_output

def mm_ss_to_seconds(min_widget, sec_widget):
    m = min_widget.value
    s = sec_widget.value
    if m is None and s is None:
        return None
    if m is None:
        m = 0
    if s is None:
        s = 0
    return int(m) * 60 + int(s)

def hh_mm_ss_to_seconds(h_widget, m_widget, s_widget):
    h = h_widget.value
    m = m_widget.value
    s = s_widget.value
    if h is None and m is None and s is None:
        return None
    h = 0 if h is None else h
    m = 0 if m is None else m
    s = 0 if s is None else s
    return int(h) * 3600 + int(m) * 60 + int(s)

def classificar_corredor(volume_semanal_km, melhor_5k_segundos):
    nivel_por_volume = None
    if volume_semanal_km is None:
        nivel_por_volume = None
    elif volume_semanal_km < 20:
        nivel_por_volume = "iniciante"
    elif 20 <= volume_semanal_km <= 40:
        nivel_por_volume = "intermediario"
    else:
        nivel_por_volume = "avancado"

    nivel_por_5k = None
    if melhor_5k_segundos is not None:
        if melhor_5k_segundos > 1800:
            nivel_por_5k = "iniciante"
        elif 1200 < melhor_5k_segundos <= 1800:
            nivel_por_5k = "intermediario"
        else:
            nivel_por_5k = "avancado"

    candidatos = [nivel_por_volume, nivel_por_5k]
    candidatos = [c for c in candidatos if c is not None]
    if not candidatos:
        return "iniciante"

    if "avancado" in candidatos:
        return "avancado"
    if "intermediario" in candidatos:
        return "intermediario"
    return "iniciante"

idade_anos = BoundedIntText(description="Idade (anos)", min=10, max=90, value=30)
sexo = Dropdown(description="Sexo", options=[("Masculino", "masculino"),
                                             ("Feminino", "feminino"),
                                             ("Outro", "outro")], value="masculino")
altura_cm = BoundedFloatText(description="Altura (cm)", min=100, max=230, value=175.0)
peso_kg = BoundedFloatText(description="Peso (kg)", min=30, max=200, value=70.0)
profissao_atividade = Dropdown(
    description="Atividade profissional",
    options=[("Sedentário", "sedentario"),
             ("Moderadamente ativo", "moderadamente_ativo"),
             ("Altamente ativo", "altamente_ativo")],
    value="moderadamente_ativo"
)
horas_sono_noite = BoundedFloatText(description="Horas sono/noite", min=0, max=14, value=7.0)
estresse_diario = BoundedIntText(description="Estresse (0-10)", min=0, max=10, value=5)

box_identificacao = VBox([
    Label("1. Identificação"),
    idade_anos, sexo, altura_cm, peso_kg,
    profissao_atividade, horas_sono_noite, estresse_diario
])

tempo_corrida_meses = BoundedIntText(description="Meses correndo", min=0, max=600, value=6)
volume_semanal_km = BoundedFloatText(description="Volume semanal (km)", min=0, max=300, value=20.0)
treinos_semana = BoundedIntText(description="Treinos/semana", min=0, max=14, value=3)

ritmo_facil_seg_km = BoundedIntText(description="Ritmo fácil (seg/km)", min=120, max=900, value=360)
ritmo_moderado_seg_km = BoundedIntText(description="Ritmo moderado (seg/km)", min=120, max=900, value=330)

melhor_5k_min = BoundedIntText(description="5k (min)", min=0, max=59, value=25)
melhor_5k_sec = BoundedIntText(description="5k (seg)", min=0, max=59, value=0)

melhor_10k_min = BoundedIntText(description="10k (min)", min=0, max=180, value=50)
melhor_10k_sec = BoundedIntText(description="10k (seg)", min=0, max=59, value=0)

melhor_21k_h = BoundedIntText(description="21k (h)", min=0, max=10, value=1)
melhor_21k_m = BoundedIntText(description="21k (min)", min=0, max=59, value=50)
melhor_21k_s = BoundedIntText(description="21k (seg)", min=0, max=59, value=0)

melhor_42k_h = BoundedIntText(description="42k (h)", min=0, max=10, value=3)
melhor_42k_m = BoundedIntText(description="42k (min)", min=0, max=59, value=30)
melhor_42k_s = BoundedIntText(description="42k (seg)", min=0, max=59, value=0)

provas_12_meses = Dropdown(
    description="Provas últimos 12m",
    options=[("Não", False), ("Sim", True)],
    value=False
)

box_historico_corrida = VBox([
    Label("2. Histórico de corrida"),
    tempo_corrida_meses,
    volume_semanal_km,
    treinos_semana,
    ritmo_facil_seg_km,
    ritmo_moderado_seg_km,
    HBox([melhor_5k_min, melhor_5k_sec]),
    HBox([melhor_10k_min, melhor_10k_sec]),
    HBox([melhor_21k_h, melhor_21k_m, melhor_21k_s]),
    HBox([melhor_42k_h, melhor_42k_m, melhor_42k_s]),
    provas_12_meses
])

lesao_tendinite = Checkbox(description="Tendinite", value=False)
lesao_fascite = Checkbox(description="Fascite", value=False)
lesao_itbs = Checkbox(description="ITBS", value=False)
lesao_dor_lombar = Checkbox(description="Dor lombar", value=False)
lesao_fratura_estresse = Checkbox(description="Fratura estresse", value=False)
lesao_outra = Checkbox(description="Outra lesão", value=False)

frequencia_lesoes_ano = BoundedIntText(description="Lesões/ano", min=0, max=20, value=0)
condicao_medica = Dropdown(
    description="Condição médica",
    options=[("Nenhuma", "nenhuma"),
             ("Cardíaca", "cardiaca"),
             ("Respiratória", "respiratoria"),
             ("Metabólica", "metabolica"),
             ("Outra", "outra")],
    value="nenhuma"
)
usa_medicacao = Dropdown(
    description="Usa medicação",
    options=[("Não", False), ("Sim", True)],
    value=False
)

box_lesoes_saude = VBox([
    Label("3. Histórico de lesões e saúde"),
    HBox([lesao_tendinite, lesao_fascite, lesao_itbs]),
    HBox([lesao_dor_lombar, lesao_fratura_estresse, lesao_outra]),
    frequencia_lesoes_ano,
    condicao_medica,
    usa_medicacao
])

fc_reposo_bpm = BoundedIntText(description="FC repouso (bpm)", min=30, max=120, value=60)
fc_max_bpm = BoundedIntText(description="FC máx (bpm)", min=120, max=230, value=190)

teste_1k_min = BoundedIntText(description="1k (min)", min=0, max=59, value=4)
teste_1k_sec = BoundedIntText(description="1k (seg)", min=0, max=59, value=0)

teste_3k_min = BoundedIntText(description="3k (min)", min=0, max=59, value=14)
teste_3k_sec = BoundedIntText(description="3k (seg)", min=0, max=59, value=0)

teste_5k_min = BoundedIntText(description="5k teste (min)", min=0, max=59, value=25)
teste_5k_sec = BoundedIntText(description="5k teste (seg)", min=0, max=59, value=0)

disp_fc = Checkbox(description="Disp. FC", value=True)
disp_potencia = Checkbox(description="Disp. potência", value=False)
disp_cadencia = Checkbox(description="Disp. cadência", value=True)
disp_gps = Checkbox(description="Disp. GPS", value=True)

box_dados_fisio = VBox([
    Label("4. Dados fisiológicos e testes"),
    fc_reposo_bpm,
    fc_max_bpm,
    HBox([teste_1k_min, teste_1k_sec]),
    HBox([teste_3k_min, teste_3k_sec]),
    HBox([teste_5k_min, teste_5k_sec]),
    HBox([disp_fc, disp_potencia, disp_cadencia, disp_gps])
])

objetivo_principal = Dropdown(
    description="Objetivo principal",
    options=[("Saúde", "saude"),
             ("Performance", "performance"),
             ("Emagrecimento", "emagrecimento"),
             ("Prova específica", "prova_especifica")],
    value="performance"
)
meta_distancia = Dropdown(
    description="Meta distância",
    options=[("5 km", "5k"),
             ("10 km", "10k"),
             ("21 km", "21k"),
             ("42 km", "42k"),
             ("Outra", "outra")],
    value="10k"
)
reducao_tempo_min = BoundedFloatText(description="Redução alvo (min)", min=0, max=120, value=0.0)
data_proxima_prova = Dropdown(
    description="Data prova",
    options=[("Não definida", ""), ("Exemplo: 2025-12-31", "2025-12-31")],
    value=""
)
motivacao_0_10 = BoundedIntText(description="Motivação (0-10)", min=0, max=10, value=8)

box_objetivos = VBox([
    Label("5. Objetivos do atleta"),
    objetivo_principal,
    meta_distancia,
    reducao_tempo_min,
    data_proxima_prova,
    motivacao_0_10
])

dias_treino_semana = BoundedIntText(description="Dias treino/sem", min=0, max=14, value=4)
tempo_max_sessao_min = BoundedFloatText(description="Tempo máx sessão (min)", min=0, max=300, value=60.0)
acesso_academia = Checkbox(description="Acesso à academia", value=True)
preferencia_treino = Dropdown(
    description="Pref. treino",
    options=[("Curto", "curto"),
             ("Longo", "longo"),
             ("Intervalado", "intervalado"),
             ("Limiar", "limiar"),
             ("Sem preferência", "sem_preferencia")],
    value="sem_preferencia"
)
aderencia_historica = Dropdown(
    description="Aderência histórica",
    options=[("Alta", "alta"), ("Média", "media"), ("Baixa", "baixa")],
    value="alta"
)

box_restricoes = VBox([
    Label("6. Restrições e disponibilidade"),
    dias_treino_semana,
    tempo_max_sessao_min,
    acesso_academia,
    preferencia_treino,
    aderencia_historica
])

nivel_percebido = BoundedIntText(description="Nível percebido (0-10)", min=0, max=10, value=5)
disponibilidade_energetica = BoundedIntText(description="Disp. energética (0-10)", min=0, max=10, value=7)
tolerancia_intenso = BoundedIntText(description="Tol. treino intenso (0-10)", min=0, max=10, value=6)
contraindicacao_alta_intensidade = Checkbox(description="Contraindicação alta intensidade", value=False)

box_classificacao_auto = VBox([
    Label("7. Classificação automática - percepções"),
    nivel_percebido,
    disponibilidade_energetica,
    tolerancia_intenso,
    contraindicacao_alta_intensidade
])

botao_salvar = Button(description="Salvar dados e classificar", button_style="success")
output = Output()

def on_salvar_clicked(b):
    with output:
        clear_output()
        melhor_5k_segundos = mm_ss_to_seconds(melhor_5k_min, melhor_5k_sec)
        melhor_10k_segundos = mm_ss_to_seconds(melhor_10k_min, melhor_10k_sec)
        melhor_21k_segundos = hh_mm_ss_to_seconds(melhor_21k_h, melhor_21k_m, melhor_21k_s)
        melhor_42k_segundos = hh_mm_ss_to_seconds(melhor_42k_h, melhor_42k_m, melhor_42k_s)

        teste_1k_segundos = mm_ss_to_seconds(teste_1k_min, teste_1k_sec)
        teste_3k_segundos = mm_ss_to_seconds(teste_3k_min, teste_3k_sec)
        teste_5k_segundos = mm_ss_to_seconds(teste_5k_min, teste_5k_sec)

        dados = {
            "identificacao": {
                "idade_anos": idade_anos.value,
                "sexo": sexo.value,
                "altura_cm": float(altura_cm.value),
                "peso_kg": float(peso_kg.value),
                "profissao_atividade": profissao_atividade.value,
                "horas_sono_noite": float(horas_sono_noite.value),
                "estresse_diario_0_10": estresse_diario.value
            },
            "historico_corrida": {
                "tempo_corrida_meses": tempo_corrida_meses.value,
                "volume_semanal_km": float(volume_semanal_km.value),
                "treinos_semana": treinos_semana.value,
                "ritmo_facil_seg_km": ritmo_facil_seg_km.value,
                "ritmo_moderado_seg_km": ritmo_moderado_seg_km.value,
                "melhor_5k_segundos": melhor_5k_segundos,
                "melhor_10k_segundos": melhor_10k_segundos,
                "melhor_21k_segundos": melhor_21k_segundos,
                "melhor_42k_segundos": melhor_42k_segundos,
                "provas_12_meses": bool(provas_12_meses.value)
            },
            "historico_lesoes_saude": {
                "lesao_tendinite": bool(lesao_tendinite.value),
                "lesao_fascite": bool(lesao_fascite.value),
                "lesao_itbs": bool(lesao_itbs.value),
                "lesao_dor_lombar": bool(lesao_dor_lombar.value),
                "lesao_fratura_estresse": bool(lesao_fratura_estresse.value),
                "lesao_outra": bool(lesao_outra.value),
                "frequencia_lesoes_ano": frequencia_lesoes_ano.value,
                "condicao_medica": condicao_medica.value,
                "usa_medicacao": bool(usa_medicacao.value)
            },
            "dados_fisiologicos_testes": {
                "fc_reposo_bpm": fc_reposo_bpm.value,
                "fc_max_bpm": fc_max_bpm.value,
                "teste_1k_segundos": teste_1k_segundos,
                "teste_3k_segundos": teste_3k_segundos,
                "teste_5k_segundos": teste_5k_segundos,
                "disp_frequencia_cardiaca": bool(disp_fc.value),
                "disp_potencia": bool(disp_potencia.value),
                "disp_cadencia": bool(disp_cadencia.value),
                "disp_gps": bool(disp_gps.value)
            },
            "objetivos": {
                "objetivo_principal": objetivo_principal.value,
                "meta_distancia": meta_distancia.value,
                "reducao_tempo_minutos": float(reducao_tempo_min.value),
                "data_proxima_prova": data_proxima_prova.value,
                "motivacao_0_10": motivacao_0_10.value
            },
            "restricoes_disponibilidade": {
                "dias_treino_semana": dias_treino_semana.value,
                "tempo_max_sessao_min": float(tempo_max_sessao_min.value),
                "acesso_academia": bool(acesso_academia.value),
                "preferencia_treino": [preferencia_treino.value],
                "aderencia_historica": aderencia_historica.value
            },
            "classificacao_automatica": {
                "nivel_percebido_0_10": nivel_percebido.value,
                "disponibilidade_energetica_0_10": disponibilidade_energetica.value,
                "tolerancia_intenso_0_10": tolerancia_intenso.value,
                "contraindicacao_alta_intensidade": bool(contraindicacao_alta_intensidade.value)
            }
        }

        classificacao = classificar_corredor(
            volume_semanal_km=float(volume_semanal_km.value),
            melhor_5k_segundos=melhor_5k_segundos
        )
        dados["rotulo_classificacao"] = classificacao

        with open("corredor_dados.json", "w", encoding="utf-8") as f:
            json.dump(dados, f, ensure_ascii=False, indent=2)

        print("Dados salvos em 'corredor_dados.json'.")
        print(f"Classificação automática do corredor: **{classificacao}**")

botao_salvar.on_click(on_salvar_clicked)

formulario = VBox([
    box_identificacao,
    box_historico_corrida,
    box_lesoes_saude,
    box_dados_fisio,
    box_objetivos,
    box_restricoes,
    box_classificacao_auto,
    botao_salvar,
    output
])

display(formulario)
